In [4]:
# Importing required libraries
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import Lasso
from sklearn.metrics import mean_absolute_percentage_error
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LassoLarsIC
from sklearn.pipeline import make_pipeline
import time
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score
from sklearn.linear_model import LinearRegression
import numpy as np

In [6]:
# Importing the dataset
Autompg = pd.read_csv("auto_mpg.csv", index_col = 0)
Autompg.head()

,mpg,cylinders,displacement,horsepower,weight,acceleration,model year,origin
0,18.0,8,307.0,130.0,3504,12.0,70,1
1,15.0,8,350.0,165.0,3693,11.5,70,1
2,18.0,8,318.0,150.0,3436,11.0,70,1
3,16.0,8,304.0,150.0,3433,12.0,70,1
4,17.0,8,302.0,140.0,3449,10.5,70,1


**Question 1 - Train data is 80%**

In [209]:
# Selecting specific columns for analysis
Autompg_new = Autompg.loc[:,Autompg.columns!='mpg']
Autompg_new

,cylinders,displacement,horsepower,weight,acceleration,model year,origin
0,8,307.0,130.0,3504,12.0,70,1
1,8,350.0,165.0,3693,11.5,70,1
2,8,318.0,150.0,3436,11.0,70,1
3,8,304.0,150.0,3433,12.0,70,1
4,8,302.0,140.0,3449,10.5,70,1
...,...,...,...,...,...,...,...
393,4,140.0,86.0,2790,15.6,82,1
394,4,97.0,52.0,2130,24.6,82,2
395,4,135.0,84.0,2295,11.6,82,1
396,4,120.0,79.0,2625,18.6,82,1


In [211]:
# Assigning the independent and dependent variables
x = Autompg_new
y = Autompg.mpg

In [214]:
# Splitting the data into the training and testing sets
x_train,x_test,y_train,y_test=train_test_split(x,y,test_size=0.20,random_state=0)

In [215]:
# Defining the Lasso Model
model_lasso2 = Lasso(alpha=1.0)

In [216]:
# Fitting the Lasso model
model_lasso2.fit(x_train,y_train)

Lasso()

In [217]:
Y_bar = y_test.mean()

In [218]:
# Predictions from the model
pred = model_lasso2.predict(x_test)

In [219]:
# Calculating the residual deviance
Residual_deviance = sum(( y_test - pred )**2)
Residual_deviance

892.8062709592677

In [220]:
# Calculating the null deviance
Null_deviance = sum(( y_test - Y_bar )**2)
Null_deviance

5076.59

In [221]:
# Out of sample R-square 
1 - (Residual_deviance/Null_deviance)

0.8241326813945449

In [222]:
# The Mean absolute percentage Error for the original and predicted values 
mean_absolute_percentage_error(y_test, pred)

0.12099427120261268

**Train data is 20%**

In [77]:
# Splitting the data into the training and testing sets
x_train1,x_test1,y_train1,y_test1=train_test_split(x,y,test_size=0.80,random_state=0)

In [223]:
# Defining the Lasso Model
model_lasso1 = Lasso(alpha=1.0)

In [224]:
# Fitting the Lasso model
model_lasso1.fit(x_train1,y_train1)

Lasso()

In [225]:
Y_bar1 = y_test1.mean()

In [226]:
# Predictions from the model
pred1 = model_lasso1.predict(x_test1)

In [227]:
Residual_deviance1 = sum(( y_test1 - pred1 )**2)
Residual_deviance1

4069.6288704122203

In [228]:
Null_deviance1 = sum(( y_test1 - Y_bar1 )**2)
Null_deviance1

19854.211974921636

In [229]:
# Out of sample R-square 
1 - (Residual_deviance1/Null_deviance1)

0.7950244071357416

In [230]:
# The Mean absolute percentage Error for the original and predicted values 
mean_absolute_percentage_error(y_test1, pred1)

0.11777526594806227

**Question 2**

In [272]:
start_time = time.time()
lasso_lars_ic = make_pipeline(
    StandardScaler(), LassoLarsIC(criterion="aic", normalize=False)
).fit(x_train, y_train)
fit_time = time.time() - start_time

In [273]:
results = pd.DataFrame(
    {
        "alphas": lasso_lars_ic[-1].alphas_,
        "AIC criterion": lasso_lars_ic[-1].criterion_,
    }
).set_index("alphas")
alpha_aic = lasso_lars_ic[-1].alpha_

In [274]:
lasso_lars_ic.set_params(lassolarsic__criterion="bic").fit(x_train, y_train)
results["BIC criterion"] = lasso_lars_ic[-1].criterion_
alpha_bic = lasso_lars_ic[-1].alpha_

In [277]:
def highlight_min(x):
    x_min = x.min()
    return ["font-weight: bold" if v == x_min else "" for v in x]


results.style.apply(highlight_min)

,AIC criterion,BIC criterion
alphas,,
6.395922862853813,318.000000,318.000000
4.322556057532109,202.770742,206.532793
3.5635049331065414,170.892484,178.416587
1.6757425539633188,91.997013,103.283167
0.4216611643792746,68.930417,83.978623
0.06247425193198665,68.914675,87.724932
0.031362929453618904,70.095176,92.667484
0.0,71.591202,97.925561


Thus looking at the results we select the value of alpha as 0.4216611643792746 because the value of BIC is lowest at this value of alpha. We are selecting the value of alpha based on the lowest value of BIC because this is a large dataset and hence BIC is a better measure as compared to AIC, because BIC provides better accuracy in determining the best model when the dataset is large.

In [235]:
# model with the optimal value of alpha
model_lasso = Lasso(alpha=0.4216611643792746)

In [236]:
model_lasso.fit(x_train,y_train)
Y_barl = y_test.mean()
predl = model_lasso.predict(x_test)
Residual_deviancel = sum(( y_test - predl )**2)
Null_deviancel = sum(( y_test - Y_barl )**2)

# Out of sample R-square
1 - (Residual_deviancel/Null_deviancel)

0.8256445380478143

**K-fold cross Validation**

In [237]:
# Cross - validation
cv = KFold(n_splits=10, random_state=1, shuffle=True)

In [238]:
# Simple Linear regression
model_linear = LinearRegression()

In [239]:
scores = cross_val_score(model_linear, x, y, scoring='r2', cv=cv, n_jobs=-1)

In [240]:
print('The r-squares for all the different models are')
scores

The r-squares for all the different models are


array([0.87457426, 0.84945464, 0.81752589, 0.83400233, 0.74159183,
       0.80590281, 0.82383345, 0.78638523, 0.77581887, 0.79600731])

In [271]:
print('The average r-square for all the models is')
scores.mean()

The average r-square for all the models is


0.8105096627069838

Thus we conclude from observing the out of sample r-squares for all the models and the average r-square after cross validation for the simple regression model that the Lasso model with tuned alpha is better than all the other models because the r-square for it is the highest. 

**Calculating the AIC, BIC and AICc**

In [242]:
# Lasso Coefficients
model_lasso.coef_

array([-0.        ,  0.00481336, -0.0182385 , -0.0062437 ,  0.05567346,
        0.71199182,  0.37867321])

In [243]:
k=6
predlt = model_lasso.predict(x_train)

In [244]:
# AIC
len(x_train)*np.log(sum(( y_train - predlt )**2)/len(x_train)) + (2*k)

789.3948588645806

In [245]:
# BIC
len(x_train)*np.log(sum(( y_train - predlt )**2)/len(x_train)) + (np.log(len(x_train))*k)

811.9671671612616

In [246]:
#AICc
len(x_train)*np.log(sum(( y_train - predlt )**2)/len(x_train)) + 2*k*(len(x_train)/(len(x_train)-k-1))

789.664955327603

***The AIC BIC and AICc for the model with 80% train data in Question 1***

In [257]:
# Lasso coefficients 
model_lasso2.coef_

array([-0.00000000e+00,  2.50099929e-04, -1.91603225e-02, -6.04260420e-03,
        0.00000000e+00,  6.61157463e-01,  0.00000000e+00])

In [250]:
k1=4
predt = model_lasso2.predict(x_train)

In [252]:
# AIC
len(x_train)*np.log(sum(( y_train - predt )**2)/len(x_train)) + (2*k1)

799.0225366847842

In [254]:
# BIC
len(x_train)*np.log(sum(( y_train - predt )**2)/len(x_train)) + (np.log(len(x_train))*k1)

814.0707422159049

In [256]:
#AICc
len(x_train)*np.log(sum(( y_train - predt )**2)/len(x_train)) + 2*k1*(len(x_train)/(len(x_train)-k1-1))

799.1503322119407

***The AIC BIC and AICc for the model with 20% train data in Question 1***

In [259]:
model_lasso1.coef_

array([-0.        ,  0.00137872, -0.01147584, -0.00627184,  0.        ,
        0.59405214,  0.        ])

In [265]:
k2 = 4
predt1 = model_lasso1.predict(x_train1)

In [266]:
# AIC
len(x_train1)*np.log(sum(( y_train1 - predt1 )**2)/len(x_train1)) + (2*k2)

187.09261721007204

In [268]:
# BIC
len(x_train1)*np.log(sum(( y_train1 - predt1 )**2)/len(x_train1)) + (np.log(len(x_train1))*k2)

196.5704086199401

In [270]:
#AICc
len(x_train1)*np.log(sum(( y_train1 - predt1 )**2)/len(x_train1)) + 2*k2*(len(x_train1)/(len(x_train1)-k2-1))

187.63315775061258